<a href="https://colab.research.google.com/github/SampadSikder/Nonnegative-Matrix-Factorization-Experimentation/blob/main/accuracy_of_coil_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Equations from https://proceedings.neurips.cc/paper/2000/file/f9d1152547c0bde01830b7e8bd60024c-Paper.pdf

In [1]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
from sklearn.cluster import KMeans
import numpy
import matplotlib.pyplot as mtp
import os
from sklearn.decomposition import NMF
import collections

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Code Library

In [3]:
import time

def extractFrameFromImage(filePath, imageLimit, frame_rate):
  vidcap = cv2.VideoCapture(filePath)
  success,image = vidcap.read()
  count = 0
  #define framerate
  prev = 0
  frame=0
  while success:
    time_elapsed = time.time() - prev
    success,image = vidcap.read()  
    if time_elapsed > 1./frame_rate:
      prev = time.time()
      cv2.imwrite("%d.jpg" % count, image)     # save frame as JPEG file      
      #print('Read a new frame: ', success)
      count=count+1
      frame += frame_rate # i.e. at 30 fps, this advances one second
      vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame)
    if count>=imageLimit:
      break
  vidcap.release()
  cv2.destroyAllWindows()

  

In [4]:
def countWidthandHeight(imagePath, downscalingFactor):
  width,height=0,0;
  for img in os.listdir(imagePath):
    extension = os.path.splitext(img)[1][1:]
    if extension=='jpg' or extension=='png':
      sampleImagePath=imagePath+'/'+img
      img=cv2.imread(sampleImagePath)
      image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
      #resize

      width = int(image.shape[1]/downscalingFactor)
      height = int(image.shape[0]/downscalingFactor)

      print(width, height)

      break


  count=0

  for image_name in os.listdir(imagePath):
    extension = os.path.splitext(image_name)[1][1:]
    if extension=='jpg' or extension=='png':
      count=count+1

  return width, height, count

In [5]:
def convertAllImageToMatrix(obj, width, height, imageCount, imageRootDir): # imageCount = number of images  , imageRootDir = "/content"

  y = width*height

  print(imageCount) 

  m = np.zeros(shape=(imageCount, int(y)))
  print(m.shape)
  itr = 0

  cntImage={}

  for index, image in enumerate(obj):
    cntImage[obj[index]]=0

  name_list = []
  for image_name in os.listdir(imageRootDir): 
    extension = os.path.splitext(image_name)[1][1:]
    if extension=='jpg' or extension=='png':
      image_label=image_name.split("__")[0]
      if image_label in obj and cntImage[image_label]<50:
        cntImage[image_label]+=1
        image_path=imageRootDir+'/'+image_name
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #resize
        dim = (width, height)
        print(width, height)
        #resizing image
        resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        
        rows,cols=resized.shape#normal shape
        img_size = rows*cols
        img_1D_vector = resized.reshape(img_size)
        
        m[itr]=img_1D_vector
        itr=itr+1
        name_list.append(image_name)
        print(itr)

        if itr>=imageCount:
          break
      
  return m, name_list, itr

In [6]:
def normalizeM(m):
  sum=0
  row, col=m.shape
  for i in range(0,row):
    for j in range(0,col):
      sum+=m[i][j]
  mean=sum/row*col 

  m=m/mean

  return m, mean

In [7]:
def updateW(x,w,h):
  #XVT
  ht=np.transpose(h)
  numerator=np.matmul(x,ht)
  #UVVT
  den1=np.matmul(w,h)
  denominator=np.matmul(den1,ht)

  #return
  print('U completed#########################################################################################')
  secondPart=np.divide(numerator,denominator)
  return np.multiply(w,secondPart)

In [8]:
def updateH(x,w,h):
  #UT*X
  wt=np.transpose(w)
  numerator=np.matmul(wt,x)
  #UTUV
  den1=np.matmul(w,h)
  denominator=np.matmul(wt,den1)
  
  #update return
  print('V completed#########################################################################################')
  secondPart=np.divide(numerator,denominator)
  return np.multiply(h,secondPart)

In [9]:
def performNMF(x, featureNo, imageCount, EPSILON=1e-9):
  w=np.random.normal(3,2.5,size=(imageCount,featureNo))
  h=np.random.normal(3,2.5,size=(featureNo, x.shape[1]))

  print("Shape of W: "+str(w.shape))
  print("Shape of H: "+str(h.shape))

  itr=0
  cost=np.linalg.norm(x-np.matmul(w,h))
  while cost>EPSILON:
    if itr%2==0:
      h=updateH(x,w,h)
    else:
      w=updateW(x,w,h)
    wh=np.matmul(w,h)
    print(w,h)
    cost=np.linalg.norm(x-wh)
    print(cost)
    if itr>=100:
      break
    itr=itr+1
  return w,h

### Testing Code

In [10]:
from google.colab.patches import cv2_imshow

In [11]:
def getImageListFromM(x_new):
  images = []
  for row in x_new:
    img_data = []
    for i in range(height):
      img_row = []
      for j in range(width):
        img_row.append(row[i*width + j])
      img_data.append(img_row)
    images.append(img_data)
  return images

In [12]:
def compareAndTest(m, w, h, mean):
  x_new=np.matmul(w,h)
  x_new=np.ceil(x_new*mean)
  #x_new=np.transpose(x_new)

  resulting_images = getImageListFromM(x_new)
  original_images = getImageListFromM(np.ceil(m*mean))

  for index in range(m.shape[0]):
    print("-------------------------------Number of image: "+str(index))

    img=np.array(resulting_images[index])
    cv2_imshow(img)
    print("Image after NMF:")

    img=np.array(original_images[index])
    cv2_imshow(img)
    print("Image Before NMF:")


In [13]:
def variance(m): # m = matrix of all images
  return np.var(m)

In [14]:
def __grouper(labels, original_images, resulting_images):
    groups = []
    for current_label in set(labels):
      cluster = []
      for index,label in enumerate(labels):
        if label == current_label:
          node = [label, original_images[index], resulting_images[index]]
          cluster.append(node)
      groups.append(cluster)
    return groups

In [15]:
def compareAndTestWithKNN(m, w, h, mean, n_clusters):
  x_new=np.matmul(w,h)
  x_new=np.ceil(x_new*mean)
  #x_new=np.transpose(x_new)

  kmeans = KMeans(n_clusters, random_state=0)
  kmeans.fit(w)

  resulting_images = getImageListFromM(x_new)
  original_images = getImageListFromM(np.ceil(m*mean))
  
  clusters = __grouper(kmeans.labels_, original_images, resulting_images)

  for cluster in clusters:
    print("----------------Labels------------------------")
    for index, node in enumerate(cluster):
      print("-------------------------------Number of image: "+str(index)+"-----------Cluster="+str(node[0])+".")
      img=np.array(node[1])
      cv2_imshow(img)
  
  return kmeans

In [16]:
def scatterPlot(kmeans,F):
  y_predict= kmeans.fit_predict(F)  
  mtp.scatter(F[y_predict == 0, 0], F[y_predict == 0, 1], s = 10, c = 'blue', label = 'Cluster 1') #for first cluster  
  mtp.scatter(F[y_predict == 1, 0], F[y_predict == 1, 1], s = 10, c = 'green', label = 'Cluster 2') #for second cluster  
  mtp.scatter(F[y_predict== 2, 0], F[y_predict == 2, 1], s = 10, c = 'red', label = 'Cluster 3') #for third cluster  
  mtp.scatter(F[y_predict == 3, 0], F[y_predict == 3, 1], s = 10, c = 'cyan', label = 'Cluster 4') #for fourth cluster  
  mtp.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s = 30, c = 'yellow', label = 'Centroid')   
  mtp.title('Clusters')  

  mtp.legend()  
  mtp.show() 

# Experimentation Area

In [89]:
def testSingleFrameMatrix(image, width, height, kmeans, nmf_model):
  image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

  dim = (width, height)
  resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
  
  rows,cols=resized.shape
  img_size = rows*cols
  img_1D_vector = resized.reshape(img_size)
  print(img_1D_vector.shape)
  img_transformed = nmf_model.transform([img_1D_vector])
  print(img_transformed.shape)
  prediction = kmeans.predict(img_transformed)[0]

  return prediction


In [90]:
def getLabels(imageNameList):
  labels = {}
  for name in imageNameList:
    label = name.split("__")[0]
    labels[name] = label

  return labels

In [91]:
def setObjects(path,numberOfObjects):
  obj=[]
  cnt=0
  for image_name in os.listdir(path): 
    extension = os.path.splitext(image_name)[1][1:]
    if extension=='jpg' or extension=='png':
      label = image_name.split("__")[0]
      obj.append(label)
      cnt+=1
    if cnt>=numberOfObjects:
      break
  
  return obj


In [92]:
def assignLabel(imageNameList, kmeans):
  labels = {}
  for i in range(len(imageNameList)):
    imageName, _ = imageNameList[i].split("__")
    labels.setdefault(imageName, []).append(kmeans.labels_[i])

  
  for key, value in labels.items():
    count = collections.Counter(value)
    labels[key] = count.most_common(1)[0][0]

  return labels

In [93]:
def train(path,n_components, n_clusters): # It requires the images to be in /content
  imageLimit = 2000
  frame_rate=5
  #extractFrameFromImage(path, imageLimit, frame_rate)
  width, height, imageCount=countWidthandHeight(path, 7)
  
  obj= setObjects(path,n_clusters)


  m, imageNameList, imageCount = convertAllImageToMatrix(obj, width, height, imageLimit, path)

  print(m.shape)
  labels=getLabels(imageNameList)
  
  #m, mean = normalizeM(m)
  mean=1
  #w,h=performNMF(m, n_components, len(imageNameList))
  nmf_model = NMF(n_components,max_iter=5000, init= 'nndsvd')
  w = nmf_model.fit_transform(m)
  h=nmf_model.components_

  print(w.shape)
  print(h.shape)
  x_new=np.matmul(w,h)
  x_new=np.ceil(x_new*mean)
  x_new=np.transpose(x_new)
  #print("Original Image Set Variance: "+str(variance(np.ceil(np.transpose(m)*mean))))
  #print("NMF Generated Image Set Variance: "+str(variance(x_new)))
  
  kmeans = KMeans(n_clusters, random_state=2)
  kmeans.fit(w)

  print(kmeans.labels_)
  labels=assignLabel(imageNameList,kmeans)

  return width, height, m, w, h, labels, kmeans, imageNameList, nmf_model
  

In [94]:
def test(testPath, width, height, labels, kmeans, nmf_model,imageNameList):
   correct=0
   failed=0
   for image_name in os.listdir(testPath):
    print(image_name) 
    extension = os.path.splitext(image_name)[1][1:]
    check_label=image_name.split("__")[0]
    if extension=='jpg' or extension=='png':
      if image_name not in imageNameList and check_label in labels.keys():
        print(image_name)
        image_path=testPath+'/'+image_name
        image = cv2.imread(image_path) 
        test_label=testSingleFrameMatrix(image, width, height, kmeans, nmf_model)
        actual_label=labels[check_label]
        print("---------------------------")
        print("Label from kmeans prediction: ",test_label)
        print("Actual label: ",actual_label)
        
        if test_label==actual_label:
          correct+=1
        else:
          failed+=1

        print(correct, failed)

   return (correct/(correct+failed)*100)
  

In [95]:
width, height, m,w,h,labels,kmeans,imageNameList, nmf_model=train('/content/coil-100/',49,100)

18 18
2000
(2000, 324)
18 18
1
18 18
2
18 18
3
18 18
4
18 18
5
18 18
6
18 18
7
18 18
8
18 18
9
18 18
10
18 18
11
18 18
12
18 18
13
18 18
14
18 18
15
18 18
16
18 18
17
18 18
18
18 18
19
18 18
20
18 18
21
18 18
22
18 18
23
18 18
24
18 18
25
18 18
26
18 18
27
18 18
28
18 18
29
18 18
30
18 18
31
18 18
32
18 18
33
18 18
34
18 18
35
18 18
36
18 18
37
18 18
38
18 18
39
18 18
40
18 18
41
18 18
42
18 18
43
18 18
44
18 18
45
18 18
46
18 18
47
18 18
48
18 18
49
18 18
50
18 18
51
18 18
52
18 18
53
18 18
54
18 18
55
18 18
56
18 18
57
18 18
58
18 18
59
18 18
60
18 18
61
18 18
62
18 18
63
18 18
64
18 18
65
18 18
66
18 18
67
18 18
68
18 18
69
18 18
70
18 18
71
18 18
72
18 18
73
18 18
74
18 18
75
18 18
76
18 18
77
18 18
78
18 18
79
18 18
80
18 18
81
18 18
82
18 18
83
18 18
84
18 18
85
18 18
86
18 18
87
18 18
88
18 18
89
18 18
90
18 18
91
18 18
92
18 18
93
18 18
94
18 18
95
18 18
96
18 18
97
18 18
98
18 18
99
18 18
100
18 18
101
18 18
102
18 18
103
18 18
104
18 18
105
18 18
106
18 18
107
18 18
108
18 18

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[ 0 63 33 ... 47 30 54]


In [96]:
print(labels)

{'obj62': 0, 'obj32': 63, 'obj25': 33, 'obj66': 12, 'obj80': 85, 'obj6': 89, 'obj42': 22, 'obj7': 0, 'obj4': 27, 'obj30': 19, 'obj94': 64, 'obj87': 4, 'obj85': 10, 'obj93': 0, 'obj91': 72, 'obj33': 36, 'obj67': 72, 'obj23': 22, 'obj48': 77, 'obj55': 73, 'obj54': 87, 'obj10': 33, 'obj51': 62, 'obj98': 29, 'obj8': 89, 'obj35': 15, 'obj12': 51, 'obj46': 20, 'obj14': 71, 'obj28': 5, 'obj20': 99, 'obj83': 16, 'obj36': 38, 'obj2': 77, 'obj92': 8, 'obj79': 20, 'obj95': 9, 'obj34': 25, 'obj71': 11, 'obj40': 20, 'obj56': 23, 'obj50': 17, 'obj74': 40, 'obj18': 30, 'obj17': 41, 'obj70': 34, 'obj39': 39, 'obj45': 68, 'obj31': 67, 'obj29': 63, 'obj44': 29, 'obj5': 7, 'obj11': 47, 'obj64': 50, 'obj61': 8, 'obj16': 86, 'obj65': 6, 'obj53': 53, 'obj63': 27, 'obj52': 26, 'obj38': 22, 'obj58': 21, 'obj37': 18}


In [97]:

print(kmeans.labels_)

[ 0 63 33 ... 47 30 54]


In [98]:
accuracy=test("/content/coil-100", width, height, labels, kmeans, nmf_model, imageNameList)

Streaming output truncated to the last 5000 lines.
1179 772
obj92__100.png
obj92__100.png
(324,)
(1, 49)
---------------------------
Label from kmeans prediction:  75
Actual label:  8
1179 773
obj86__180.png
obj15__230.png
obj73__295.png
obj43__45.png
obj3__140.png
obj94__140.png
obj94__140.png
(324,)
(1, 49)
---------------------------
Label from kmeans prediction:  64
Actual label:  64
1180 773
obj33__310.png
obj33__310.png
(324,)
(1, 49)
---------------------------
Label from kmeans prediction:  3
Actual label:  36
1180 774
obj52__50.png
obj52__50.png
(324,)
(1, 49)
---------------------------
Label from kmeans prediction:  26
Actual label:  26
1181 774
obj11__10.png
obj11__10.png
(324,)
(1, 49)
---------------------------
Label from kmeans prediction:  64
Actual label:  47
1181 775
obj93__215.png
obj93__215.png
(324,)
(1, 49)
---------------------------
Label from kmeans prediction:  0
Actual label:  0
1182 775
obj15__260.png
obj26__140.png
obj35__290.png
obj35__290.png
(324,)
(1, 

In [99]:

print(accuracy)

60.44952681388013


In [25]:

!mv coil-100/* /contentb zx
!ls /content

mv: target 'zx' is not a directory
drive  sample_data


In [ ]:
!ls /content/drive/MyDrive/Colab\ Notebooks/NMF\ datasets/COIL\ 100/coil-100/coil-100/
!cp /content/drive/MyDrive/Colab\ Notebooks/NMF\ datasets/COIL\ 100/coil-100/coil-100/ . -r
!ls coil-100